In [0]:
#cd "drive/My Drive/Colab Notebooks/dw_matrix/matrix_one"

In [0]:
#!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score as cvs

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values
  
  scores = cvs(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [133]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
df['brand'] = df['brand'].map(lambda x: str(x).lower())
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [134]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [135]:
str_dict = df.features[0]
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [136]:
df['features'].map(lambda x: type(x)).value_counts()

<class 'str'>      13299
<class 'float'>     4981
Name: features, dtype: int64

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  
  
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    output_dict[item['key'].lower().strip()] = item['value'][0].lower().strip()

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [140]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [141]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [142]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_country of origin', 'feat_country of origin components:',
       'feat_resizable', 'feat_neckline', 'feat_autographed', 'feat_brand_cat',
       'feat_color_cat', 'feat_gender_cat',
       'feat_manufacturer part number_cat', 'feat_material_cat'],
      dtype='object', length=531)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0]/ df.shape[0]  * 100

In [144]:
{k:v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) +'_cat'] = df[get_name_feat(key)].factorize()[0]


In [146]:
feats= ['feat_brand_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feats)

(-61.79232381882688, 3.7460167781236606)

In [0]:
feats= ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat','feat_sport_cat', 'feat_style_cat','feat_shape_cat','feat_metal type_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [189]:


X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit (X, y)


print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

(-58.52170810232822, 4.236276263829771)


Weight,Feature
0.2558 ± 0.0080,brand_cat
0.1035 ± 0.0163,feat_material_cat
0.0249 ± 0.0039,feat_gender_cat
0.0178 ± 0.0007,feat_brand_cat
0.0128 ± 0.0004,feat_shape_cat
0.0083 ± 0.0014,feat_metal type_cat
0.0035 ± 0.0008,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [190]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   HelloGithub.ipynb
	modified:   matrix_one/Day3.ipynb
	modified:   matrix_one/day4.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
